# Lab 10
Understanding Docker Networks

What is Container Networking?
Container networking also called Software Defined Networking (SDN) is what allows Docker networkgin subsystems to work. 

Some defalt Docker Netwrok drivers are:
- Bridge
    - a Network that is User-defined to allow for all containers connected on the same network to communicate with eachother. 
- Host
    - a container is attahed to a Docker host. Any traffic routed to the container must be routed from the host system.
- Overlay
    - Created in a network spanning multiple docker hosts. Think each of your docker containers in the lab talking to eachother
- Macvlan
    - used in complicated networking to assing logical addresses based on MAC addresses to the virtual network interfaces making it apear as if the container has a physical network interaface connected to the network. Used in legacy applications.
- None
    -  the container only has a loopback interface so it can talk to itself.



# Part 1 - Setup

In [6]:
!docker run -d --name nginx --network=none -p 81:81 nginx

7c1312e85329ecfd7bd34a38c81231d08500ee0a1b1297421c00cf6bb6436efc


## Note:
**If you get this error:**

*docker: permission denied while trying to connect to the Docker daemon ...*

run this in wsl: 
``` bash
sudo usermod -aG docker $USER
newgrp docker
sudo systemctl restart docker
```
Run it again....

you should get a hash number YAY!

`cbb2f01cd1b...`

--------------------------------------
If you are still getting the error


run this:
``` bash
sudo chmod 666 /usr/docker.sock
```
--------------------------------------
**If you get this error you did things right**

docker: Error response from daemon: Conflict. The container name "/nginx

--------------------------------------


## we now have an nginx container 

NGINX serves static content (like HTML, CSS, JavaScript, images) efficiently, improving website performance.

In [2]:
!curl localhost

#the curl command should fail on our system 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 80 after 2262 ms: Could not connect to server


Lets run it on the docker container 

In [3]:
!docker exec -it nginx2 /bin/sh -c "curl google.com"



Error response from daemon: No such container: nginx2


* curl:(6) Could not resolve host: google.com

As you can see the nginx container can not talk to google.com

# Part 2 - Understanding the network 
Docker comes with a subcommand for handling Docker networks.

In [7]:
!docker network

Usage:  docker network COMMAND

Manage networks

Commands:
  connect     Connect a container to a network
  create      Create a network
  disconnect  Disconnect a container from a network
  inspect     Display detailed information on one or more networks
  ls          List networks
  prune       Remove all unused networks
  rm          Remove one or more networks

Run 'docker network COMMAND --help' for more information on a command.


running `docker network ls` will show us all the names of the networks.


In [8]:
!docker network ls 

NETWORK ID     NAME          DRIVER    SCOPE
9431e18b5030   bridge        bridge    local
d7a70153b638   host          host      local
f0088b586a3d   none          null      local
c357700c7f28   yourNetwork   bridge    local


In the above you can see that there is a none network. 

When we setup the docker nginx server we did not give it a network

lets look for the nginx server

In [9]:
!docker network inspect none 

[
    {
        "Name": "none",
        "Id": "f0088b586a3d4fa6bddfd0a6d326321be86174a0dbf44e2aad3f6960ed619079",
        "Created": "2025-04-12T00:39:59.680986238Z",
        "Scope": "local",
        "Driver": "null",
        "EnableIPv4": true,
        "EnableIPv6": false,
        "IPAM": {
            "Driver": "default",
            "Options": null,
            "Config": null
        },
        "Internal": false,
        "Attachable": false,
        "Ingress": false,
        "ConfigFrom": {
            "Network": ""
        },
        "ConfigOnly": false,
        "Containers": {
            "7c1312e85329ecfd7bd34a38c81231d08500ee0a1b1297421c00cf6bb6436efc": {
                "Name": "nginx",
                "EndpointID": "6c7cd0bc49c2a5fe272f45710a58395d77f86c4007b1d45277bb3b3a89346b36",
                "MacAddress": "",
                "IPv4Address": "",
                "IPv6Address": ""
            }
        },
        "Options": {},
        "Labels": {}
    }
]


## OK Enough of that nonsense
Lets install docker and use the API to do some work this is a python class after all

In [10]:
!python get-pip.py
!pip install docker

python: can't open file 'c:\\Users\\OGrex\\OneDrive - Cal Poly Pomona\\Documents\\homeworkfolder-JuanEstrada1\\Labs\\Lab 10\\get-pip.py': [Errno 2] No such file or directory


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: C:\Users\OGrex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [11]:
import docker

client = docker.from_env()

print("Docker version:", client.version()['Version'])


ModuleNotFoundError: No module named 'docker'

Lets create a quick code to list the containers using the docker client

In [12]:
def list_running_containers():
    containers = client.containers.list(all=True)# list all containers
    #containers = client.containers.list() #list only running containers
    for container in containers:
        print(f"{container.name} - {container.status}")

list_running_containers()

NameError: name 'client' is not defined

Here you can see the running container and no surprise it is the nginx we started before


In [13]:
#lest identify the network of the nginx container
def show_container_ip(container_name):
    container = client.containers.get(container_name)
    networks = container.attrs['NetworkSettings']['Networks']
    
    for net_name, net_data in networks.items():
        print(f"{container_name} in network '{net_name}' has IP: {net_data['IPAddress']}")


show_container_ip("nginx")


nginx in network 'none' has IP: 


again no surprise there is no IP address as it does not have a network. 

### ok I lied back to creating more stuff though the CLI

In [14]:
#setup another container for mysql
!docker run -d --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8
#Note the -d means detached mode so no container is using created just the MySQL database 

docker: Error response from daemon: Conflict. The container name "/mysql" is already in use by container "8376da3ec3a9badee5fdf57c35a59cded5abd7ed08a71f9ef0e7ccc13d4cc510". You have to remove (or rename) that container to be able to reuse that name.

Run 'docker run --help' for more information


## The following needs to be run in the terminal

In [5]:
!docker run -d --name adminer -p 8080:8080 adminer


0168eb0cc8a3bc8fefab5fb8d03affcf4f663f8ec090b520fc12947889f25fcf


Unable to find image 'adminer:latest' locally
latest: Pulling from library/adminer
f18232174bc9: Pulling fs layer
b1220d176a8d: Pulling fs layer
115c7a04f6c0: Pulling fs layer
0d55cff53123: Pulling fs layer
88d591373a19: Pulling fs layer
dc6d32971fca: Pulling fs layer
ffeef6b41a56: Pulling fs layer
6d95abe48138: Pulling fs layer
a80d3af78a9d: Pulling fs layer
f3fe38446507: Pulling fs layer
c6d0aade0806: Pulling fs layer
4f4fb700ef54: Pulling fs layer
e434f3dae19e: Pulling fs layer
8b847f2809a2: Pulling fs layer
93b26376816c: Pulling fs layer
52e9abf80869: Pulling fs layer
f3fe38446507: Waiting
dc6d32971fca: Waiting
c6d0aade0806: Waiting
ffeef6b41a56: Waiting
4f4fb700ef54: Waiting
93b26376816c: Waiting
52e9abf80869: Waiting
6d95abe48138: Waiting
e434f3dae19e: Waiting
a80d3af78a9d: Waiting
0d55cff53123: Waiting
88d591373a19: Waiting
b1220d176a8d: Verifying Checksum
b1220d176a8d: Download complete
f18232174bc9: Download complete
115c7a04f6c0: Download complete
f18232174bc9: Pull complete


Run the following each in there own terminal

``` bash
docker logs -f mysql
docker logs -f adminer
```

In [15]:
!docker inspect mysql | grep -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("mysql")


------------------------------
mysql in network 'yourNetwork' has IP: 


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
!docker inspect adminer| grep  -n1 "IPAddress\":"
# or
print("-"*30)
show_container_ip("adminer")


------------------------------
adminer in network 'yourNetwork' has IP: 


'grep' is not recognized as an internal or external command,
operable program or batch file.


### Lets do a bit of cleanup 

``` bash
#lets remove our old container
!docker stop adminer
!docker rm adminer
!docker stop mysql
!docker rm mysql
!docker stop nginx 
!docker rm nginx
```

In [17]:
def clean_containers():
    containers = client.containers.list(all=True)
    if not containers:
        print("No containers to clean.")
        return
    
    for container in containers:
        container.stop()
        container.reload()
        if container.status == 'exited':
            print(f"Removing {container.name}")
            container.remove()

list_running_containers()

clean_containers()


nginx - running
adminer - exited
mysql - exited
Removing nginx
Removing adminer
Removing mysql


## Setting up the network

In [18]:
# Create a network called yourNetwork using the following command

!docker network create yourNetwork

error during connect: Head "http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/_ping": open //./pipe/dockerDesktopLinuxEngine: The system cannot find the file specified.


In [2]:
!docker network inspect yourNetwork

[]


error during connect: Get "http://%2F%2F.%2Fpipe%2FdockerDesktopLinuxEngine/v1.48/networks/yourNetwork": open //./pipe/dockerDesktopLinuxEngine: The system cannot find the file specified.


In [20]:
!docker run -d --network yourNetwork --name mysql -p 3306:3306 -e MYSQL_ROOT_PASSWORD=1 mysql:8


459f16ad41ec0241296054bbe54b5fbab8cfd0863e61277384e737ebb129a603


In [21]:
!docker run -d --network yourNetwork --name adminer -p 8080:8080 adminer

f2b05b6ad52c4e714603470b5f3541fe530d0519ff32b568897884521368f5a1


In [22]:
#Now lets connect the adminer
!docker network connect yourNetwork adminer

Error response from daemon: endpoint with name adminer already exists in network yourNetwork


In [23]:
!docker inspect mysql |grep -C2 " \"Networks\""
!docker inspect mysql |grep -A9 " \"Networks\""| grep " \"IPAddress\""
#or
print("-"*30)
show_container_ip("mysql")

'grep' is not recognized as an internal or external command,
operable program or batch file.


------------------------------
mysql in network 'yourNetwork' has IP: 172.18.0.2


'grep' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
!docker inspect adminer| grep -A2 " \"Networks\""
!docker inspect adminer| grep -A9 " \"Networks\""| grep " \"IPAddress\""
!ping -c 1 172.18.0.3
#or
print("-"*30)
show_container_ip("adminer")

'grep' is not recognized as an internal or external command,
operable program or batch file.
'grep' is not recognized as an internal or external command,
operable program or batch file.


Access denied. Option -c requires administrative privileges.
------------------------------


NameError: name 'show_container_ip' is not defined

# LAB 
As a reminder you should be submitting to me 2 files. 
- _YourName_Lab 09.ipynb
    - this is your lab report
- _YourName__\<name of your script\>.py

## Part 1 - warm up

Now that you have 2 containers up and running and know there connected. Create a python script that will check to see if the IP addresses of the containers are up and running. 



In [3]:
import docker
import subprocess
import platform

client = docker.from_env()

def get_container_ip(container_name):
    try:
        container = client.containers.get(container_name)
        networks = container.attrs['NetworkSettings']['Networks']
        for net_name, net_data in networks.items():
            return net_data['IPAddress']
        return None
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found.")
        return None
    except Exception as e:
        print(f"Error getting IP for '{container_name}': {e}")
        return None
    
def check_running_containers(container_names):
    for container_name in container_names:
        ip = get_container_ip(container_name)
        if ip:
            print(f"Checking container '{container_name}' with IP {ip}...")
        else:
            print(f"Could not retrieve IP for '{container_name}'.")

def main():
    container_names = ["mysql", "adminer"]
    check_running_containers(container_names)

if __name__ == "__main__":
    main()

get_container_ip("adminer")
get_container_ip("mysql")

ModuleNotFoundError: No module named 'docker'


## Part 2
Continue with the above code and create a script that will:
- Relaunch a container if it is ever shutdown. 
- Perform a maintenance restart after 1 day has passed. 
- Monitor the log files and outputs errors and warnings.

your python file does not need to be in a docker container


https://docs.docker.com/engine/api/sdk/examples/



In [ ]:
import docker
import subprocess
import platform
import time
from datetime import datetime, timedelta

client = docker.from_env()

def get_container_ip(container_name):
    try:
        container = client.containers.get(container_name)
        networks = container.attrs['NetworkSettings']['Networks']
        for net_name, net_data in networks.items():
            return net_data['IPAddress']
        return None
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found.")
        return None
    except Exception as e:
        print(f"Error getting IP for '{container_name}': {e}")
        return None
    
def check_running_containers(container_names):
    for container_name in container_names:
        ip = get_container_ip(container_name)
        if ip:
            print(f"Checking container '{container_name}' with IP {ip}...")
        else:
            print(f"Could not retrieve IP for '{container_name}'.")

def restart_container(container_name):
    try:
        container = client.containers.get(container_name)
        container.restart()
        print(f"Container '{container_name}' restarted at {datetime.now()}")
        return datetime.now()
    except docker.errors.NotFound:
        print(f"Container '{container_name}' not found, cannot restart.")
        return None
    except Exception as e:
        print(f"Error restarting '{container_name}': {e}")
        return None

def monitor_containers(container_names, maintenance_interval_hours=24):
    last_restarts = {name: datetime.now() for name in container_names}
    interval = timedelta(hours=maintenance_interval_hours)
    
    while True:
        for name in container_names:
            status = get_container_ip(name)
            if status is None:
                continue
            
            needs_restart = status != "running" or (datetime.now() - last_restarts[name]) >= interval
            if needs_restart:
                reason = "not running" if status != "running" else "maintenance"
                print(f"Restarting '{name}' ({reason})...")
                restart_time = restart_container(name)
                if restart_time:
                    last_restarts[name] = restart_time
        
        time.sleep(60)

def main():
    container_names = ["mysql", "adminer"]
    check_running_containers(container_names)

if __name__ == "__main__":
    main()

get_container_ip("adminer")
get_container_ip("mysql")

ModuleNotFoundError: No module named 'docker'

In [1]:
pip install pdfkit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: C:\Users\OGrex\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip
